In [5]:
import pinocchio as pin
import numpy as np
import time
import scipy
from example_robot_data import load

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

## visualise the polytope and the ellipsoid
import meshcat.geometry as g 

# import pycapacity 
import pycapacity as pycap

import meshcat

In [6]:
def computeCollisions(robot, geom_model, q, verbose = True):
    geom_data = geom_model.createData()
    has_collision = pin.computeCollisions(robot.model, robot.data, geom_model, geom_data, q, not verbose)
    if verbose:
        print("Has collision {}".format(has_collision))
        i = 0
        for k in range(len(geom_model.collisionPairs)): 
          cr = geom_data.collisionResults[k]
          cp = geom_model.collisionPairs[k]
          if cr.isCollision():
              i = i+1
              print("collision pair:",cp.first,",",cp.second,"- collision:","Yes" if cr.isCollision() else "No")
        print(i)
    return has_collision

In [7]:
# use reachy's right arm with only 4 joints for now as they are the contribute 
# to its force capacity the most
urdf_path = "reachy_v3_fix.urdf"
robot = pin.RobotWrapper.BuildFromURDF(urdf_path)
model, data = robot.model, robot.data
# lock the other joints
tolock = [
   "l_shoulder_pitch",
   "l_shoulder_roll",
   "l_elbow_yaw",
   "l_elbow_pitch",
   "l_wrist_roll",
   "l_wrist_pitch",
   "l_wrist_yaw",
   "l_hand_finger",
   "l_hand_finger_mimic",
   "l_hand_finger_proximal",
   "l_hand_finger_distal",
   "l_hand_finger_proximal_mimic",
   "l_hand_finger_distal_mimic",
   "neck_roll",
   "neck_pitch",
   "neck_yaw",
   # "r_shoulder_pitch",
   # "r_shoulder_roll",
   # "r_elbow_yaw",
   # "r_elbow_pitch",
   # "r_wrist_roll",
   # "r_wrist_pitch",
   # "r_wrist_yaw",
   "r_hand_finger",
   "r_hand_finger_mimic",
   "r_hand_finger_proximal",
   "r_hand_finger_distal",
   "r_hand_finger_proximal_mimic",
   "r_hand_finger_distal_mimic",
]

# Get the ID of all existing joints
jointsToLockIDs = []
for jn in tolock:
    if model.existJointName(jn):
        jointsToLockIDs.append(model.getJointId(jn))
robot.model, robot.collision_model = pin.buildReducedModel(model, robot.collision_model, jointsToLockIDs, np.zeros(robot.nq))
model, data = robot.model, robot.data



In [8]:
print("available collision objects")
for i, g in enumerate(robot.collision_model.geometryObjects):
    print(i,g.name)

available collision objects
0 base_link_0
1 back_bar_0
2 torso_0
3 l_elbow_arm_link_0
4 l_elbow_forearm_link_0
5 l_wrist_link_0
6 l_hand_palm_link_0
7 l_hand_proximal_link_0
8 l_hand_distal_link_0
9 l_hand_proximal_mimic_link_0
10 l_hand_distal_mimic_link_0
11 head_0
12 r_elbow_arm_link_0
13 r_elbow_forearm_link_0
14 r_wrist_link_0
15 r_hand_palm_link_0
16 r_hand_proximal_link_0
17 r_hand_distal_link_0
18 r_hand_proximal_mimic_link_0
19 r_hand_distal_mimic_link_0
20 left_bar_0
21 right_bar_0


In [9]:
viewer = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [11]:
robot.data = robot.model.createData()

# get joint position ranges
q_max = robot.model.upperPositionLimit.T
q_min = robot.model.lowerPositionLimit.T
# q_min = np.array([-np.pi, -3.07, -np.pi, -2.22,-np.pi/4,-np.pi/4,-np.pi])[:robot.nq]
# q_max = np.array([np.pi, 0.54, np.pi, 0.02,np.pi/4,np.pi/4,np.pi])[:robot.nq]
# robot.model.upperPositionLimit = q_max
# robot.model.lowerPositionLimit = q_min
dq_max = 5*np.ones(robot.nq)
dq_min = -dq_max
# get max velocity
t_max = np.ones(robot.nq)*4 # amps
t_min = -t_max

# Use robot configuration.
q = robot.q0[:robot.nq]
# q = (q_min+q_max)/2

model = robot.model
data = robot.model.createData()
geom_model = robot.collision_model
for i in [0, 1, 2, 11, 20, 21]: # torso, head, 3x bars
    # geom_model.addCollisionPair(pin.CollisionPair(i,3)) # left upper arm
    # geom_model.addCollisionPair(pin.CollisionPair(i,4)) # left lower arm
    # geom_model.addCollisionPair(pin.CollisionPair(i,5)) # left wrist
    # geom_model.addCollisionPair(pin.CollisionPair(i,6)) # left palm
    geom_model.addCollisionPair(pin.CollisionPair(i,12)) # right upper arm
    geom_model.addCollisionPair(pin.CollisionPair(i,13)) #  right lower arm
    geom_model.addCollisionPair(pin.CollisionPair(i,14)) #right  wrist
    geom_model.addCollisionPair(pin.CollisionPair(i,15)) # left palm
geom_data = pin.GeometryData(geom_model)
robot.rebuildData()
    

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.collision_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True, viewer = viewer)
# Load the robot in the viewer.
viz.loadViewerModel("reachy")
viz.display(q)
# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 
viz.display_collisions = True

In [13]:
from ipywidgets import interact, FloatSlider, Layout, Label

kwargs = {"\tq[{}]".format(i) : 
          FloatSlider(
              min = q_min[i], 
              max = q_max[i], 
              step = 0.01, 
              value = q[i],
          layout=Layout(width='300px'))
          for i,q_1 in enumerate(q)}
@interact(**kwargs)
def update(**kwargs):
    q = np.array([v  for v in kwargs.values()])
    viz.display(q)
    computeCollisions(robot, geom_model, q)

interactive(children=(FloatSlider(value=0.0, description='\tq[0]', layout=Layout(width='300px'), max=1.5707963…